In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json

In [ ]:
with open('../src/base_config.json') as json_file:
    config_dict1 = json.load(json_file)

with open('../src/period_config.json') as json_file:
    config_dict2 = json.load(json_file)

# Merging the two config dictionaries
config_dict = {**config_dict1, **config_dict2}


In [ ]:
config_dict

In [ ]:
filename_billings = config_dict['path_to_data'] + config_dict['billings']['filename']
sheetname_billings = config_dict['billings']['base_sheetname']

In [ ]:
filename_billingsllings_billings

In [ ]:
sheetname_billings

In [ ]:
df = pd.read_excel(filename_billings, sheet_name=sheetname_billings)

In [ ]:
df.head(10)

In [ ]:
df.info()

In [2]:
config_dict  = {
    "ADBE_cal": {
        "direct_filename": "../Data/ADOBE_FINANCIAL_CALENDAR.xlsx",
        "sheetname": "ADBE_cal"
    },
    "FX_forwards": {
        "filename": "FX_forward_rates.xlsx",
        "sheetname": "forward_data"
    },
    "FX_rates": {
        "filename": "FX_data.xlsx",
        "sheetname": "to_matlab"
    },
    "billings": {
        "filename": "all_billings_inputs.xlsx",
        "base_sheetname": "base_billings",
        "type_A_sheetname": "type_A_no_config"
    },
    "bookings": {
        "filename_DME": "DME_Bookings_FY21_Plan.xlsx",
        "sheetname_DME": "Raw",
        "filename_DX": "DX_Bookings_FY21_Plan.xlsx",
        "sheetname_DX": "Sheet1",
        "start_row_DX": 12
    },
    "curr_banned": [
        "BMD",
        "MXP",
        "PLN"
    ],
    "curr_map": {
        "filename": "currency_map.xlsx",
        "sheetname": "curr_map"
    },
    "list_future_periods": [
        "2021-04",
        "2021-05",
        "2021-06",
        "2021-07",
        "2021-08",
        "2021-09",
        "2021-10",
        "2021-11",
        "2021-12",
        "2022-01",
        "2022-02",
        "2022-03"
    ],
    "path_to_data": "../Data/Data_2021_p03/",
    "output_dir": {
        "intermediate": "../Data/Data_2021_p03/processed/intermediate/",
        "final": "../Data/processed/final/"
    },
    "deferred_workbook": {
        "filename": "Q1'21 Rev Acctg Mgmt Workbook.xlsx",
        "sheetname": "Deferred Revenue Forecast",
        "skiprows": 5,
        "keeper_rows": [
            "Digital Media Total",
            "Publishing & AdCloud Total",
            "Digital Experience Total",
            "Workfront Total"
        ],
        "drop_columns": [
            "Major Product Config",
            " Historical"
        ]
    },
    "bookings_BU_mapping": {
        "Creative": "Digital Media",
        "Document Cloud": "Digital Media",
        "DX Other": "Digital Experience",
        "Experience Cloud": "Digital Experience",
        "Print & Publishing": "Publishing"
    },
    "waterfall_BU_mapping": {
        "Digital Experience Total": "Digital Experience",
        "Digital Media Total": "Digital Media",
        "Publishing & AdCloud Total": "Publishing"
    },
    "waterfall_initial_period": "2021-04"
}

In [3]:
path = config_dict["path_to_data"]
filename = config_dict['deferred_workbook']['filename']
sheetname = config_dict['deferred_workbook']['sheetname']
skiprows = config_dict['deferred_workbook']['skiprows']
path_and_filename= path + filename

df = pd.read_excel(path_and_filename,
    sheet_name=sheetname,
    skiprows=skiprows)

# some of the columns rows have blank spaces stripping this out
df['External Reporting BU'] = df['External Reporting BU'].str.strip()

# clearing out columns that we do not need
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
drop_columns = config_dict['deferred_workbook']['drop_columns']
df = df.drop(columns=drop_columns)

# clearing out rows we do not need
keeper_rows = config_dict['deferred_workbook']['keeper_rows']
df = df[df['External Reporting BU'].isin(keeper_rows)]

'''
Below was to account for Magento and Marketo deferred being listed separately. It is now in Digital Experience

Below was in teh period_config under deferred_workbook:
 "str_replace_from": [
        "Magento Deferred",
        "Marketo Deferred"
    ],
    "str_replace_to": "Digital Experience Total",

list_BU_changes = config_dict['deferred_workbook']["str_replace_from"]
list_BU_new = config_dict['deferred_workbook']["str_replace_to"]
for item in list_BU_changes:
    df["External Reporting BU"] = df["External Reporting BU"].str.replace(
        item, list_BU_new)

Q2 2021: We now have Workfront, but it is included in the 'Grand Total incl. Workfront'
We need to add the difference between the 'Grand Total' and the 'Grand Total incl. Workfront" and add to
Digital Experience
BELOW IS A ONE TIME CHANGE THAT I WILL HAVE TO MAKE INTO THE DICTIONARY AT SOME POINT

df.loc['Digital Experience Total'] = df.loc['Digital Experience Total'] + df.loc['Grand Total incl Workfront'] -
    df.loc['Grant Total']


'''

'\nBelow was to account for Magento and Marketo deferred being listed separately. It is now in Digital Experience\n\nBelow was in teh period_config under deferred_workbook:\n "str_replace_from": [\n        "Magento Deferred",\n        "Marketo Deferred"\n    ],\n    "str_replace_to": "Digital Experience Total",\n\nlist_BU_changes = config_dict[\'deferred_workbook\']["str_replace_from"]\nlist_BU_new = config_dict[\'deferred_workbook\']["str_replace_to"]\nfor item in list_BU_changes:\n    df["External Reporting BU"] = df["External Reporting BU"].str.replace(\n        item, list_BU_new)\n\nQ2 2021: We now have Workfront, but it is included in the \'Grand Total incl. Workfront\'\nWe need to add the difference between the \'Grand Total\' and the \'Grand Total incl. Workfront" and add to\nDigital Experience\nBELOW IS A ONE TIME CHANGE THAT I WILL HAVE TO MAKE INTO THE DICTIONARY AT SOME POINT\n\ndf.loc[\'Digital Experience Total\'] = df.loc[\'Digital Experience Total\'] + df.loc[\'Grand Tota

In [4]:
df.head(10)

,External Reporting BU,Q2'21,Q3'21,Q4'21,Q1'22,Q2'22,Q3'22,Q4'22,Q1'23,Q2'23+,As Performed / Upon Acceptance,Total
13,Digital Media Total,1.2126e+06,821192,494398,165834,13669.7,9205.21,6927.17,4595.9,39147,42190.6,2.80976e+06
23,Digital Experience Total,542838,359671,241313,108373,18080.5,17808.3,1331.31,2038.86,807.076,5395.75,1.29766e+06
38,Publishing & AdCloud Total,32377.1,24662.4,13199.1,5181.17,792.455,618.898,527.23,120.561,51.0145,386.046,77916
44,Workfront Total,34316.8,29985.6,22519.5,7594.59,235.5,0,0,0,0,5375.73,100028


In [5]:
df = df.replace('Workfront Total','Digital Experience Total')

In [6]:
df.head()

,External Reporting BU,Q2'21,Q3'21,Q4'21,Q1'22,Q2'22,Q3'22,Q4'22,Q1'23,Q2'23+,As Performed / Upon Acceptance,Total
13,Digital Media Total,1.212603e+06,821191.93959,494398.28445,165833.80027,13669.67063,9205.20945,6927.17106,4595.90391,39146.95912,42190.62269,2.809762e+06
23,Digital Experience Total,5.428383e+05,359670.90124,241313.44530,108373.10375,18080.50986,17808.27501,1331.31295,2038.85711,807.07638,5395.75020,1.297658e+06
38,Publishing & AdCloud Total,3.237710e+04,24662.40848,13199.11666,5181.17049,792.45462,618.89767,527.23048,120.56098,51.01445,386.04569,7.791600e+04
44,Digital Experience Total,3.431683e+04,29985.60700,22519.46700,7594.58700,235.50000,0.00000,0.00000,0.00000,0.00000,5375.72700,1.000277e+05


In [7]:
# Removing non-alphanumeric characters from column dates
changed_columns = df.columns.str.replace("'", "_")
changed_columns = changed_columns.str.replace("+", "")
df.columns = changed_columns
df.columns

Index(['External Reporting BU', 'Q2_21', 'Q3_21', 'Q4_21', 'Q1_22', 'Q2_22',
       'Q3_22', 'Q4_22', 'Q1_23', 'Q2_23', 'As Performed / Upon Acceptance',
       'Total'],
      dtype='object')

In [8]:
# Grouping by external reporting BU now that we have changed the BUs
df = df.groupby("External Reporting BU").sum()

# Adjust for the reporting in thousands (FP&A report)
df = df * 1000

# Creating the columns that have this amortization by period
#
# Note: My forecast looks at the end of period values always. The bookings forecast is quarterly,
# which we change to be a monthly (level) bookings forecast. To arrive at the value of the
# bookings forecast at the end of the first period, one period's worth (1/3) of a qarter,
# has already been booked and need to be eliminated from the dataframe.
#
# To adjust for this, the bookings dataframe will have a P0 that contains the first month's
# bookings and will be removed.
#
# This created an error the first time I tested the program that overstated the bookings,
# billings and deferred.

In [9]:
new_columns = []
for i in range(12 * 3):
    if len(str(i)) == 1:
        new_column = "P0" + str(i)
    else:
        new_column = "P" + str(i)
    new_columns.append(new_column)

qtrly_list = [col for col in df.columns if "Q" in col]

period_index = 0
for index, qtr in enumerate(qtrly_list):
    df[new_columns[period_index]] = df[qtr] / 3
    period_index += 1
    df[new_columns[period_index]] = df[qtr] / 3
    period_index += 1
    df[new_columns[period_index]] = df[qtr] / 3
    period_index += 1

# OK My periods work fine. Now I can move on to saving this and finishing the defered waterfall

df = df.loc[:, df.columns.str.contains("P")]

# ##### Now dropping P0 from the waterfall
df = df.drop("P00", axis=1)

# Changing the BUs to match billings and dropping the total row
df = df.rename(index=config_dict['waterfall_BU_mapping'])
#df = df.drop(config_dict['waterfall_drop'])

In [10]:
df.head(10)

,As Performed / Upon Acceptance,P01,P02,P03,P04,P05,P06,P07,P08,P09,...,P17,P18,P19,P20,P21,P22,P23,P24,P25,P26
External Reporting BU,,,,,,,,,,,,,,,,,,,,,
Digital Experience,10771477.20,1.923850e+08,1.923850e+08,1.298855e+08,1.298855e+08,1.298855e+08,8.794430e+07,8.794430e+07,8.794430e+07,3.865590e+07,...,5.936092e+06,4.437710e+05,4.437710e+05,4.437710e+05,6.796190e+05,6.796190e+05,6.796190e+05,2.690255e+05,2.690255e+05,2.690255e+05
Digital Media,42190622.69,4.042010e+08,4.042010e+08,2.737306e+08,2.737306e+08,2.737306e+08,1.647994e+08,1.647994e+08,1.647994e+08,5.527793e+07,...,3.068403e+06,2.309057e+06,2.309057e+06,2.309057e+06,1.531968e+06,1.531968e+06,1.531968e+06,1.304899e+07,1.304899e+07,1.304899e+07
Publishing,386045.69,1.079237e+07,1.079237e+07,8.220803e+06,8.220803e+06,8.220803e+06,4.399706e+06,4.399706e+06,4.399706e+06,1.727057e+06,...,2.062992e+05,1.757435e+05,1.757435e+05,1.757435e+05,4.018699e+04,4.018699e+04,4.018699e+04,1.700482e+04,1.700482e+04,1.700482e+04


In [ ]:
config_dict